In [1]:
# Limpar os dados
# Criar o node
# Criar a PPC
# NLIST 1 ITEMSET
# NLIST 2 ITEMSET

In [2]:
import itertools
import copy

In [3]:
support = 2
F = []
input_file = 'entrada_teste.txt'

In [5]:
def make_itemset_1(filename):
    global support
    itemset_1 = {}
    with open(filename) as finput:
        f = finput.read().split('\n')
        for line in f:
            items = line.strip().split(' ')
            for item in items:
                if item not in itemset_1:
                    itemset_1[item] = 0
                itemset_1[item] += 1
    
    for k, v in list(itemset_1.items()):
        if v < support:
            del itemset_1[k]
    return itemset_1


itemset_1 = make_itemset_1(input_file)

In [6]:
class PPCNode:
    def __init__(self):
        self.pre_order = None
        self.post_order = None
        self.count = None
        self.label = None
        self.parent = None
        self.child = None
        self.sibling = None


class FPTNode:
    def __init__(self):
        self.equivalent_items = None
        self.child_nodes = None
        self.label = None
        self.itemset = None
        self.support = None

In [7]:
def create_node(root, label):
    node = PPCNode()
    node.label = label
    node.count = 1
    node.parent = root
    return node


def insert_ppc_node(root, item):
    if root.child is None:
        node = create_node(root, item)
        root.child = node
        return root.child
    elif root.child.label == item:
        root.child.count += 1
        return root.child
    elif root.child.sibling is None:
        node = create_node(root, item)
        root.child.sibling = node
        return root.child.sibling
    else:
        current_sibling = root.child.sibling
        last_sibling = None
        while current_sibling is not None:
            if current_sibling.label == item:
                current_sibling.count += 1
                return current_sibling
            else:
                last_sibling = current_sibling
                current_sibling = current_sibling.sibling
        node = create_node(root, item)
        last_sibling.sibling = node        
        return node

In [8]:
def pre_post(root, pre=0, post=0):
    root.pre_order = pre
    post_ = post
    if root.child is not None:
        root.child.pre_order = pre + 1
        pre, post_ = pre_post(root.child, root.child.pre_order, post)
    root.post_order = post_
    if root.sibling is not None:
        root.sibling.pre_order = pre + 1
        pre, post_ = pre_post(root.sibling, root.sibling.pre_order, root.post_order + 1)
        return pre, post_
    return pre, post_ + 1    

In [11]:
def build_ppc_tree(filename, itemset_1):
    root = PPCNode()
    with open(filename) as finput:
        f = finput.read().split('\n')
        for line in f:
            items = line.strip().split(' ')            
            transaction = []
            for item in items:
                if item in list(itemset_1.keys()):
                    transaction.append((item, itemset_1[item]))
                else:
                    print('removed:', item)
            transaction.sort(key=lambda x: x[1], reverse=True)
            root_aux = root
            for item in transaction:
                root_aux = insert_ppc_node(root_aux, item[0])
    pre_post(root)
    return root


tree = build_ppc_tree(input_file, itemset_1)

removed: 7


In [12]:
def print_tree(root):
    print(root.label, root.pre_order, root.post_order)
    if root.child:
        print_tree(root.child)
    if root.sibling:
        print_tree(root.sibling)

In [13]:
def make_n_list(root, n_list={}):
    if root.label:
        if root.label not in n_list:
            n_list[root.label] = []
        n_list[root.label].append(((root.pre_order, root.post_order), root.count))
    if root.child:
        make_n_list(root.child, n_list)
    if root.sibling:
        make_n_list(root.sibling, n_list)

In [14]:
n_list = {}
make_n_list(tree, n_list)
n_list

{'2': [((1, 7), 9)],
 '3': [((2, 2), 5), ((9, 10), 2)],
 '4': [((3, 0), 2), ((7, 5), 1), ((10, 9), 2)],
 '5': [((4, 1), 2), ((5, 6), 4)],
 '1': [((6, 3), 1), ((8, 4), 1)],
 '6': [((11, 8), 2)]}

In [15]:
F = list(itemset_1.items())
F.sort(key=lambda x: x[1], reverse=True)
items_ordered = [x[0] for x in F]
items_ordered

['2', '3', '5', '4', '1', '6']

In [16]:
F

[('2', 9), ('3', 7), ('5', 6), ('4', 5), ('1', 2), ('6', 2)]

In [17]:
def NL_interserction(n_list1, n_list2, minsup):
    n_list_result = []
    for k in n_list1:
        for l in n_list2:
            if k[0][0] < l[0][0] and k[0][1] > l[0][1]:                 
                n_list_result.append((k[0], l[1]))
    d = {x:0 for x, _ in n_list_result} 
    for name, num in n_list_result: d[name] += num 
    n_list_result = list(map(tuple, d.items()))
    n_list_result = list(filter(lambda x: x[1] >= minsup, n_list_result))
    return n_list_result

In [19]:
def make_n2_list(n_list, items_ordered):
    global support
    global F
    n2_list = {}
    for i, x in enumerate(items_ordered):
        for j in range(i+1, len(items_ordered)):
            n_list_result = NL_interserction(n_list[x], n_list[items_ordered[j]], support)
            if len(n_list_result) > 0:
                key = '-'.join([x, items_ordered[j][0]])
                n2_list[key] = n_list_result
    return n2_list

In [21]:
n2_list = make_n2_list(n_list, items_ordered)
n2_list

{'2-3': [((1, 7), 5)],
 '2-5': [((1, 7), 6)],
 '2-4': [((1, 7), 3)],
 '2-1': [((1, 7), 2)],
 '3-5': [((2, 2), 2)],
 '3-4': [((2, 2), 2), ((9, 10), 2)],
 '3-6': [((9, 10), 2)],
 '5-1': [((5, 6), 2)],
 '4-6': [((10, 9), 2)]}

In [22]:
all_n_list = copy.deepcopy(n2_list)

In [23]:
def find_subsets(items, n):
    return list(itertools.combinations(items, n))


def get_all_subsets(items):
    subsets = [find_subsets(items, i) for i in range(len(items)+1)]
    subsets = ['-'.join(map(str, item)) for item in list(itertools.chain.from_iterable(subsets))]
    return subsets[1:]

In [24]:
def get_n_list(key):
    global all_n_list
    return all_n_list[key] if key in all_n_list else []

In [25]:
def building_pattern_tree(node, cad_items, parent_fit):
    global support
    global all_n_list
    global F
    
    node.equivalent_items = []
    node.child_nodes = []
    next_cad_items = []    
    p1 = node.itemset
    p1_n_list = get_n_list('-'.join(p1))
    for item in cad_items:
        p2 = [item] + p1[1:]
        p = [item] + p1
        p2_n_list = get_n_list('-'.join(p2))        
        p_n_list = NL_interserction(p2_n_list, p1_n_list, support)
        p_support = sum([item[1] for item in p_n_list])
        p_key = '-'.join(p)
        
        if len(p_n_list) > 0: # add n_list for new k-itemset            
            if p_key not in all_n_list:
                all_n_list[p_key] = []
            all_n_list[p_key] += p_n_list
            
        if p_support == node.support:
            node.equivalent_items += [item]
        elif p_support >= support:
            node_ = FPTNode()
            node_.label = item
            node_.itemset = p
            node_.support = p_support
            node.child_nodes += [node_]
            next_cad_items += [item]
    
    nd_fit = []        
    if len(node.equivalent_items) > 0:
        subsets = get_all_subsets(node.equivalent_items)
        cand_itemsets = [('-'.join([item, node.label]), node.support) for item in subsets]
        if len(parent_fit) == 0:
            nd_fit = cand_itemsets
        else:
            nd_fit = []
            for cand_item in cand_itemsets:
                for parent_item in parent_fit:
                    nd_fit.append(('-'.join([cand_item[0], parent_item[0]]), node.support))
                    
        F += nd_fit
    if len(node.child_nodes) > 0:
        for n in node.child_nodes:
            aheads_ = [i for i in items_ordered[:items_ordered.index(n.label)] if i in next_cad_items]
            if aheads_ == []:
                F.append(('-'.join(n.itemset), n.support))
            else:
                building_pattern_tree(n, aheads_, nd_fit)

In [26]:
for key, n_list in list(n2_list.items()):
    item = key[:key.index('-')]
    aheads = items_ordered[:items_ordered.index(item)]
    node = FPTNode()
    node.label = key
    node.itemset = key.split('-')
    node.support = sum([item[1] for item in n_list])
    F.append((key, node.support))
    building_pattern_tree(node, aheads, [])

In [27]:
F

[('2', 9),
 ('3', 7),
 ('5', 6),
 ('4', 5),
 ('1', 2),
 ('6', 2),
 ('2-3', 5),
 ('2-5', 6),
 ('2-4', 3),
 ('2-1', 2),
 ('3-5', 2),
 ('2-3-5', 2),
 ('3-4', 4),
 ('2-3-4', 2),
 ('3-6', 2),
 ('5-1', 2),
 ('2-5-1', 2),
 ('4-6', 2),
 ('3-4-6', 2)]

In [28]:
n2_list

{'2-3': [((1, 7), 5)],
 '2-5': [((1, 7), 6)],
 '2-4': [((1, 7), 3)],
 '2-1': [((1, 7), 2)],
 '3-5': [((2, 2), 2)],
 '3-4': [((2, 2), 2), ((9, 10), 2)],
 '3-6': [((9, 10), 2)],
 '5-1': [((5, 6), 2)],
 '4-6': [((10, 9), 2)]}

In [29]:
items_ordered

['2', '3', '5', '4', '1', '6']